In [5]:
import numpy as np
import dask
from numba import jit
import nd2
import gc

In [13]:
# del data
gc.collect()

1374

In [14]:
# Load dataset
def load_data(file:str = "../data/A1_s3001.nd2"):
    data = nd2.imread(file, dask=True)
    return data.astype("complex64")

data = load_data()


In [15]:
data = data[:1000,:,:]

In [17]:
def calc_diff(data):
    num_frames = data.shape[0]
    shift = 1

    for ii in range(num_frames-1):
        im_diff = data[ii+shift] - data[ii]

@jit
def calc_diff_jit(data):
    num_frames = data.shape[0]
    shift = 1

    for ii in range(num_frames-1):
        im_diff = data[ii+shift] - data[ii]

@jit(nopython=True, parallel=True)
def calc_diff_jit_par(data):
    num_frames = data.shape[0]
    shift = 1

    for ii in range(num_frames-1):
        im_diff = data[ii+shift] - data[ii]

In [11]:
%%timeit
calc_diff(data)

1.76 s ± 346 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
calc_diff_jit(data)

C:\Users\mauritskok\AppData\Local\Temp\ipykernel_21096\2962387967.py:8: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calc_diff_jit" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\mauritskok\AppData\Local\Temp\ipykernel_21096\2962387967.py (10)

File "..\..\..\AppData\Local\Temp\ipykernel_21096\2962387967.py", line 10:
<source missing, REPL/exec in use?>

  @jit
C:\Users\mauritskok\AppData\Local\Temp\ipykernel_21096\2962387967.py:8: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calc_diff_jit" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "..\..\..\AppData\Local\Temp\ipykernel_21096\2962387967.py", line 13:
<source missing, REPL/exec in use?>

  @jit
c:\Users\mauritskok\Anaconda3\envs\ddm_env\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Func

3.58 s ± 415 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [114]:
%%timeit
calc_diff_jit_par(data)

c:\Users\Maurits\miniconda3\envs\ddm_env\lib\site-packages\numba\core\typed_passes.py:329: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "..\..\AppData\Local\Temp\ipykernel_18596\2032226103.py", line 16:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


325 ns ± 17.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [115]:
def calc_abs(data):
    for image in data:
        im_diff = np.abs(image)**2

@jit(nopython=True)
def calc_abs_jit(data):
    for image in data:
        im_diff = np.abs(image)**2

@jit(nopython=True, parallel=True)
def calc_abs_jit_par(data):
    for image in data:
        im_diff = np.abs(image)**2

In [24]:
%%timeit
calc_abs(data)

2.66 s ± 186 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
calc_abs_jit(data)

1.82 s ± 259 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit
calc_abs_jit_par(data)

In [32]:
data_float = data.astype("float64")

C:\Users\Maurits\AppData\Local\Temp\ipykernel_18596\2437359153.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  data_float = data.astype("float64")


In [108]:
def radial_profile(data, centre):
    x, y = np.indices((data.shape))
    r = np.sqrt((x-centre[0])**2 + (y-centre[1])**2)
    r = r.astype(int)
    tbin = np.bincount(r.ravel(),data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin/nr
    return radialprofile

@jit(nopython=True, parallel=True)
def radial_profile_jit(data, centre, x, y):
    # x, y = np.indices((data.shape))
    r = np.sqrt((x-centre[0])**2 + (y-centre[1])**2)
    r = r.astype(np.int64)
    tbin = np.bincount(r.ravel(),data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin/nr
    return radialprofile

In [96]:
%%timeit
center = data.shape[1:]
for image in data_float:
    radial_profile(image, center)

C:\Users\Maurits\AppData\Local\Temp\ipykernel_18596\1392229832.py:7: RuntimeWarning: invalid value encountered in true_divide
  radialprofile = tbin/nr


8.52 s ± 492 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [109]:
%%timeit
center = (512/2, 512/2)
x, y = np.indices((data[0].shape))
for image in data_float:
    radial_profile_jit(image, center, x, y)

2.46 s ± 113 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
def get_indices(dimensions):
    N = len(dimensions)
    shape = (1,)*N
    res = np.empty((N,)+dimensions, dtype="float64")
    for i, dim in enumerate(dimensions):
            idx = np.arange(dim, dtype="float64").reshape(
                shape[:i] + (dim,) + shape[i+1:]
            )
            res[i] = idx
    return res    

@jit(nopython=True, parallel=True)
def get_indices_jit(dimensions):
    N = len(dimensions)
    # shape = (1,)*N
    shape = [1,1]
    res = np.empty((N,)+dimensions, dtype="int64")
    for i, dim in enumerate(dimensions):
            idx = np.arange(dim).reshape(
                shape[:i] + dim + shape[i+1:]
            )
            res[i] = idx
    return res    

In [51]:
%%timeit
get_indices(data[0].shape)

1.43 ms ± 57.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [74]:
%%timeit
get_indices_jit(data[0].shape)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function add>) found for signature:
 
 >>> add(list(int64)<iv=None>, int64)
 
There are 18 candidate implementations:
[1m  - Of which 16 did not match due to:
  Overload of function 'add': File: <numerous>: Line N/A.
    With argument(s): '(list(int64)<iv=None>, int64)':[0m
[1m   No match.[0m
[1m  - Of which 2 did not match due to:
  Operator Overload in function 'add': File: unknown: Line unknown.
    With argument(s): '(list(int64)<iv=None>, int64)':[0m
[1m   No match for registered cases:
    * (int64, int64) -> int64
    * (int64, uint64) -> int64
    * (uint64, int64) -> int64
    * (uint64, uint64) -> uint64
    * (float32, float32) -> float32
    * (float64, float64) -> float64
    * (complex64, complex64) -> complex64
    * (complex128, complex128) -> complex128[0m
[0m
[0m[1mDuring: typing of intrinsic-call at C:\Users\Maurits\AppData\Local

In [64]:
dimensions = data[0].shape
N = len(dimensions)
shape = (1,)*N
shape

(1, 1)